In [0]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#Para web scrapping
import re
import urllib.request as urllib2
from bs4 import BeautifulSoup
import json
import requests

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# 1. Web Scrapping

In [0]:
'''Funcion para obtener BeautifulSoupObjects'''
def getSoupObj(url):
    
    try:
        html = requests.get(url).text 
    except UnicodeEncodeError as e:
        return None
    
    return BeautifulSoup(html,'lxml')

In [0]:
final_page=10
#Cambiar el número de página
base_url="https://www.apestan.com/countries/pagenumber_{}/countryid_19/isapproved_yes/deleted_0"

**¿Qué información queremos extraer?**

- Título
- Empresa (Cía)
- Fecha
- Ubicación
- Categoría
- Número de vistas
- Texto

In [0]:
all_incidents = []

for i in range(1,final_page+1):
    page = str(i)
    
    # Construimos el link con el nùmero de página
    url = base_url.format(page)
    # BeautifulSoupObj con los links a todas las quejas en la página
    page_BsObj = getSoupObj(url)
    # Extraemos la lista de quejas, cada una está en un <div class="incident">
    incidents = page_BsObj.find("div", id="content").find_all("div", class_="incident")
    
    # Por cada incidente
    for ins in incidents:
        #link al artículo completo
        i_url = "https://www.apestan.com" + ins.find("div", class_="heading-4").a['href']
        incidentObj = getSoupObj(i_url)
        
        incident = {}
        #Título del incidente
        incident['title'] = incidentObj.find('div', class_="case_details").find_all('div', class_="border_bottom")[1].div.h1.text
        #Empresa 
        incident['company'] = incidentObj.find("span", class_="count").a.text
        #Fecha
        date_items = incidentObj.find("span", class_="right").find_all("a")
        incident['date'] = pd.Timestamp(date_items[0].text + '/' + date_items[1].text + '/' + date_items[2].text)
        #Ubicación
        locs = incidentObj.find('span', class_="small").find_all('a')
        incident['location'] = locs[0].text + ',' + locs[1].text
        #Categoría
        cats = incidentObj.find('div', class_='case_details_left').p.find_all('a')
        incident['categories'] = ''
        for c in cats: incident['categories'] = incident['categories'] + c.text + ','
        incident['categories'] = incident['categories'][:-1]
        #Número de vistas
        incident['viewers'] = incidentObj.find('span', class_="viewed").text
        #Texto
        textblocks = incidentObj.find('div', class_='text-block').find_all('span', id='', class_='')
        incident['text'] = ''
        for block in textblocks: incident['text'] = incident['text'] + block.text
        incident['text'] = incident['text'].replace('  ', ' ')
        
        all_incidents.append(incident)

In [0]:
df_incidents = pd.DataFrame(all_incidents)
df_incidents.shape

(300, 7)

In [0]:
df_incidents.sample(5)

,title,company,date,location,categories,viewers,text
17,Sally beauty Arequipa apestan,Sally Beauty,2019-05-16,"Arequipa,Arequipa","Productos,Cosméticos, Joyería y Relojes",779,Compré 01 polvo bodygraphy el 02.05 y recién ...
56,Universidad Inca Garcilaso de la Vega apesta,Universidad Inca Garcilaso de la Vega,2017-02-25,"Lima,Lima","Educación y Formación,Universidades",1594,Hola : \nEstudio en la modalidad a distancia ...
64,Indecopi Burocracia Institucionalizada,Indecopi,2016-06-12,"San Borja,Lima","Gobierno y Política,Autoridades Servidores Púb...",768,Me sorprende que no haya quejas de este organ...
147,NYCY Soporte Empresarial mentirosos estafadores,NYCY Soporte Empresarial,2014-06-17,"Breña,Lima","Trabajos,Reclutamiento",2106,"NYCY Soporte Empresarial, Esta pseudo empresa..."
19,Nido Sonrisas en la huerta Rimac abusivos,Sonrisas,2019-07-05,"Rimac,Lima","Educación y Formación,Colegios, Escuelas y Aca...",812,"Ese nido Sonrisas se pasó de abusivo, me han ..."


# 2. Pre-procesamiento

In [0]:
# Vocabulario original
from collections import Counter
c_tokens = Counter(' '.join(df_incidents.text).split(' '))
print(len(c_tokens))
c_tokens.most_common(15)

14950


[('de', 3234),
 ('que', 2978),
 ('y', 1997),
 ('la', 1673),
 ('el', 1494),
 ('a', 1463),
 ('en', 1312),
 ('no', 1170),
 ('me', 851),
 ('los', 820),
 ('por', 741),
 ('un', 699),
 ('se', 697),
 ('con', 658),
 ('para', 645)]

**Signos de puntuación**

In [0]:
#Remover signos de puntuacion
import string 

def remove_punctuation(sentence: str):
    return sentence.translate(str.maketrans('', '', string.punctuation))

In [0]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [0]:
remove_punctuation(df_incidents.text[0])

' Buenos días les escribo desde Perú ayer 01052020 compré un panel de créditos a la empresa iptvlatino Link me recibieron el dinero pero hasta el día de hoy no me entregaron el panel no contestan los mensajes del correo soporteIptvlatinocommx me eliminaron del Whatsapp desde el cual me comunicaba con ellos \n a través de una señorita ES 52166244893xx Las autoridades mexicanas deberían hacer algo al respecto ya que esta seudo empresa viene estafando a muchas personas y desde hace años'

**Stopwords**

In [0]:
#Remover números y stopwords
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
stop_words = stopwords.words('spanish')
print(stop_words[:15])

['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las', 'por', 'un', 'para', 'con']


In [0]:
def remove_stopwords(sentence: str):
    tokens = [word for word in nltk.word_tokenize(sentence.lower()) if (word not in stop_words)]
    return ' '.join(tokens)

In [0]:
remove_stopwords(df_incidents.text[0])

'buenos días , escribo perú , ayer 01/05/2020 compré panel créditos empresa iptvlatino ( link recibieron dinero día hoy entregaron panel , contestan mensajes correo ( soporte @ iptvlatino.com.mx ) , eliminaron whatsapp comunicaba , través señorita e.s ( 52166244893xx ) . autoridades mexicanas deberían hacer respecto , seudo empresa viene estafando muchas personas hace años .'

**Remover números**

In [0]:
def remove_numbers(sentence: str):
    tokens = [word for word in nltk.word_tokenize(sentence.lower()) if re.search('[a-zA-Z]', word)]
    return ' '.join(tokens)

In [0]:
remove_numbers(df_incidents.text[0])

'buenos días les escribo desde perú ayer compré un panel de créditos a la empresa iptvlatino link me recibieron el dinero pero hasta el día de hoy no me entregaron el panel no contestan los mensajes del correo soporte iptvlatino.com.mx me eliminaron del whatsapp desde el cual me comunicaba con ellos a través de una señorita e.s 52166244893xx las autoridades mexicanas deberían hacer algo al respecto ya que esta seudo empresa viene estafando a muchas personas y desde hace años'

In [0]:
def text_preprocesor(sentence: str):
    #remover puntuación
    sentence = sentence.translate(str.maketrans('', '', string.punctuation))
    #remover stopwords y números
    tokens = [word for word in nltk.word_tokenize(sentence.lower()) if (word not in stop_words) and re.search('[a-zA-Z]', word)]
    return ' '.join(tokens)

In [0]:
text_preprocesor(df_incidents.text[0])

'buenos días escribo perú ayer compré panel créditos empresa iptvlatino link recibieron dinero día hoy entregaron panel contestan mensajes correo soporteiptvlatinocommx eliminaron whatsapp comunicaba través señorita 52166244893xx autoridades mexicanas deberían hacer respecto seudo empresa viene estafando muchas personas hace años'

In [0]:
df_incidents['text_prep'] = df_incidents['text'].apply(text_preprocesor)

In [0]:
c_tokens = Counter(' '.join(df_incidents.text_prep).split(' ')) 
print(len(c_tokens))
c_tokens.most_common(15)

10325


[('si', 273),
 ('mas', 183),
 ('empresa', 181),
 ('servicio', 141),
 ('solo', 130),
 ('nunca', 124),
 ('dinero', 118),
 ('hace', 108),
 ('personas', 107),
 ('hacer', 98),
 ('cuenta', 87),
 ('ahora', 85),
 ('ser', 84),
 ('gente', 79),
 ('banco', 78)]

In [0]:
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("spanish")

def steam_text(sentence: str):
    tokens = [stemmer.stem(word) for word in nltk.word_tokenize(sentence.lower())]
    return ' '.join(tokens)

In [0]:
df_incidents['text_steam'] = df_incidents['text_prep'].apply(steam_text)

In [0]:
c_tokens = Counter(' '.join(df_incidents.text_steam).split(' ')) 
print(len(c_tokens))
c_tokens.most_common(15)

5863


[('hac', 329),
 ('si', 273),
 ('pag', 269),
 ('sol', 236),
 ('empres', 222),
 ('mas', 184),
 ('person', 179),
 ('llam', 177),
 ('trabaj', 168),
 ('servici', 162),
 ('pas', 162),
 ('compr', 155),
 ('mes', 142),
 ('asi', 142),
 ('dic', 140)]

# 3. POS-Tagging

In [0]:
!pip install -U spacy
!python -m spacy download es_core_news_sm

Requirement already up-to-date: spacy in /usr/local/lib/python3.6/dist-packages (2.2.4)
     |████████████████████████████████| 16.2MB 2.7MB/s 
  Created wheel for es-core-news-sm: filename=es_core_news_sm-2.2.5-cp36-none-any.whl size=16172936 sha256=ec2b76aa1fe47f4ee0c470ce6fcf0cad51ad22e166e7291420b546516d63b115
  Stored in directory: /tmp/pip-ephem-wheel-cache-i2150spz/wheels/05/4f/66/9d0c806f86de08e8645d67996798c49e1512f9c3a250d74242
Successfully built es-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_sm')


In [0]:
import spacy
nlp = spacy.load("es_core_news_sm")

In [0]:
from tqdm import tqdm

all_pos = []
for sent in tqdm(df_incidents['text_prep']):
    doc = nlp(sent)
    for token in doc: all_pos.append(token.pos_)

In [0]:
c_tokens = Counter(all_pos) 
print(len(c_tokens))
c_tokens.most_common(15)